In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from utils import MyTabNetClassifier
from acv_explainers.utils import *
from ares import AReS
from cet import CounterfactualExplanationTree
from ce import ActionExtractor
from utils import DatasetHelper, submodular_picking, DATASETS_NAME
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
# Load Diabetes dataset

seed = 2022
np.random.seed(0)

X = pd.read_csv('data/diabetes/train.csv')
y = X.Outcome

X.drop(['Id', 'Outcome'], axis=1, inplace=True)

# X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.20, random_state=seed)

In [3]:
# Train IsolationForest

from sklearn.ensemble import IsolationForest

isolation = IsolationForest()
isolation.fit(X_tr)

IsolationForest()

In [4]:
y_tr = 1 - y_tr
y_ts = 1 - y_ts

In [5]:
# Train the model to explain 

mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
mdl.fit(X_tr, y_tr)

LGBMClassifier(n_estimators=50, num_leaves=8)

In [6]:
feature_names = [col.replace(' ', '') for col in X_tr.columns]
feature_names

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [25]:
target_name = 'Diabetes'
target_labels = ['Good', 'Bad']

In [26]:
feature_constraints = ['' for i in range(len(feature_names))]
len(feature_constraints) == X_tr.shape[1]

True

In [27]:
feature_categories = []

In [28]:
# feature_types = ['C' for i in range(len(feature_names))]
feature_types = 5*['I'] + 2*['C'] + ['I']
feature_types

['I', 'I', 'I', 'I', 'I', 'C', 'C', 'I']

In [11]:
X_tr = X_tr.values
X_ts = X_ts.values

In [12]:
y_tr = y_tr.values
y_ts = y_ts.values

In [13]:
X = X_tr[mdl.predict(X_tr)==1]
X_vl = X_ts[mdl.predict(X_ts)==1]

## Actionable Recourse Summary

In [15]:
print('## Actionable Recourse Summary')
ares = AReS(mdl, X_tr, max_rule=4, max_rule_length=4, discretization_bins=10, minimum_support=0.05, print_objective=False,
            feature_names=feature_names, feature_types=feature_types, feature_categories=feature_categories, 
            feature_constraints=feature_constraints, target_name=target_name, target_labels=target_labels)
ares = ares.fit(X, max_change_num=4, cost_type='MPS', lambda_acc=1.0, lambda_cov=1.0, lambda_cst=1.0)
print('* Parameters:')
print('\t* lambda_acc: {}'.format(ares.lambda_acc)); print('\t* lambda_cov: {}'.format(ares.lambda_cov)); print('\t* lambda_cst: {}'.format(ares.lambda_cst));
print('\t* minimum support: {}'.format(ares.rule_miner_.minsup_)); print('\t* discretization bins: {}'.format(ares.rule_miner_.fd_.bins)); print('\t* pre-processing time[s]: {}'.format(ares.preprocess_time_)); 
print('\t* max rule: {}'.format(ares.max_rule_)); print('\t* max rule length: {}'.format(ares.max_rule_length_)); print('\t* Time[s]:', ares.time_); 
print('\t* uncover test: {}'.format(ares.uncover(X_vl))); print('\t* conflict: {}'.format(ares.conflict(X_vl))); print();
print('### Learned AReS')
print(ares.to_markdown())

## Actionable Recourse Summary
* Parameters:
	* lambda_acc: 1.0
	* lambda_cov: 1.0
	* lambda_cst: 1.0
	* minimum support: 0.05
	* discretization bins: 10
	* pre-processing time[s]: 0.014364740000019083
	* max rule: 4
	* max rule length: 4
	* Time[s]: 27.056685822000873
	* uncover test: 0.6
	* conflict: 0.0

### Learned AReS
| | Rule | Action |
| :---: | --- | --- |
| Recourse <br> rule 1 <br> (probability: 11.6%) | If '84.0<=x_1<92.0' | x_1>=173.0 |
| Recourse <br> rule 2 <br> (probability: 12.2%) | If '111.0<=x_1<119.0' | x_1>=173.0 |
| Recourse <br> rule 3 <br> (probability: 12.9%) | If '99.0<=x_1<105.0' | x_1>=173.0 |
| Recourse <br> rule 4 <br> (probability: 11.4%) | If '105.0<=x_1<111.0' | x_1>=173.0 |
| Default <br> rule | Else | x_1>=173.0 |



In [29]:
x_ares = ares.predict(X_vl) + X_vl
y_ares = mdl.predict(x_ares)

print('Negative accuracy of AReS = {}'.format(np.mean(y_ares != mdl.predict(X_vl))))
print('Negative Inlier AReS = {}'.format(np.mean(isolation.predict(x_ares) == 1)))

Negative accuracy of AReS = 0.6057142857142858
Negative Inlier AReS = 0.8571428571428571


In [30]:
print('Sparsity', np.mean(np.sum(x_ares-X_vl!=0, axis=1)))

Sparsity 1.0


# Counterfactual TREE

In [19]:
print('## Counterfactual Explanation Tree')
cet = CounterfactualExplanationTree(mdl, X_tr, y_tr, max_iteration=500, lime_approximation=False,
                                    feature_names=feature_names, feature_types=feature_types, feature_categories=feature_categories,
                                    feature_constraints=feature_constraints, target_name=target_name, target_labels=target_labels)

cet = cet.fit(X, max_change_num=8, cost_type='MPS', C=LAMBDA, gamma=GAMMA, max_leaf_size=-1, time_limit=180, verbose=True)
print('* Parameters:'); print('\t* lambda: {}'.format(cet.lambda_)); print('\t* gamma: {}'.format(cet.gamma_)); print('\t* max_iteration: {}'.format(cet.max_iteration_));
print('\t* leaf size bound:', cet.leaf_size_bound_); print('\t* leaf size:', cet.n_leaves_); print('\t* LIME approximation:', cet.lime_approximation_); print('\t* Time[s]:', cet.time_); print();
print('### Learned CET')
cet.print_tree()

## Counterfactual Explanation Tree
## Stochastic Local Searching ...
### Iteration: 10
#### Before:
- If Glucose<108:
	* Action [Diabetes: Bad -> Good] (409/409 = 100.0% / MeanCost = 0.682):
		* Pregnancies: +2
		* Glucose: +54
		* SkinThickness: +28
		* BMI: +33.8520
		* DiabetesPedigreeFunction: +0.4684
		* Age: +13
- Else:
	* Action [Diabetes: Bad -> Good] (357/369 = 96.7% / MeanCost = 0.6):
		* Pregnancies: +3
		* Glucose: +5
		* SkinThickness: +27
		* BMI: +24.1800
		* DiabetesPedigreeFunction: +0.2710

#### After "Insert"
- If Glucose<108:
	* Action [Diabetes: Bad -> Good] (409/409 = 100.0% / MeanCost = 0.682):
		* Pregnancies: +2
		* Glucose: +54
		* SkinThickness: +28
		* BMI: +33.8520
		* DiabetesPedigreeFunction: +0.4684
		* Age: +13
- Else:
	- (+) If SkinThickness<35:
		* (+) Action [Diabetes: Bad -> Good] (295/299 = 98.7% / MeanCost = 0.645):
			* Pregnancies: +3
			* Glucose: +5
			* SkinThickness: +23
			* BMI: +35.4640
			* DiabetesPedigreeFunction: +0.2710
	- Else:
		* 

### Iteration: 40
#### Before:
- If Age<23:
	- If DiabetesPedigreeFunction<0.455:
		- If DiabetesPedigreeFunction<0.2242:
			* Action [Diabetes: Bad -> Good] (47/47 = 100.0% / MeanCost = 0.799):
				* Pregnancies: +11
				* Glucose: +48
				* SkinThickness: +33
				* BMI: +40.3000
				* DiabetesPedigreeFunction: +0.5152
				* Age: +21
		- Else:
			* Action [Diabetes: Bad -> Good] (75/81 = 92.6% / MeanCost = 0.666):
				* Pregnancies: +2
				* Glucose: +24
				* SkinThickness: +22
				* BMI: +37.0760
				* DiabetesPedigreeFunction: +0.3642
				* Age: +6
	- Else:
		* Action [Diabetes: Bad -> Good] (61/65 = 93.8% / MeanCost = 0.606):
			* Pregnancies: +3
			* Glucose: +45
			* BloodPressure: -20
			* SkinThickness: +29
			* BMI: +24.1800
			* DiabetesPedigreeFunction: +0.7302
			* Age: +6
- Else:
	- If Pregnancies<1:
		- If Age<26:
			* Action [Diabetes: Bad -> Good] (35/35 = 100.0% / MeanCost = 0.479):
				* Pregnancies: +1
				* Glucose: +24
				* SkinThickness: +20
				* BMI: +35.4640


- If Age<23:
	- If BloodPressure<82:
		* Action [Diabetes: Bad -> Good] (171/171 = 100.0% / MeanCost = 0.672):
			* Pregnancies: +4
			* Glucose: +48
			* SkinThickness: +20
			* BMI: +37.0760
			* DiabetesPedigreeFunction: +0.3279
			* Age: +9
	- Else:
		* Action [Diabetes: Bad -> Good] (0/22 = 0.0% / MeanCost = 0.0):
- Else:
	- If Pregnancies<1:
		- If Age<26:
			* Action [Diabetes: Bad -> Good] (35/35 = 100.0% / MeanCost = 0.479):
				* Pregnancies: +1
				* Glucose: +24
				* SkinThickness: +20
				* BMI: +35.4640
				* DiabetesPedigreeFunction: +0.1764
				* Age: +4
		- Else:
			* Action [Diabetes: Bad -> Good] (31/36 = 86.1% / MeanCost = 0.375):
				* Glucose: +14
				* BloodPressure: -4
				* SkinThickness: +15
				* BMI: +34.7080
				* DiabetesPedigreeFunction: -0.0422
				* Age: -2
	- Else:
		- If Pregnancies<2:
			- If BloodPressure<68:
				- If Insulin<70:
					- (+) If DiabetesPedigreeFunction<0.687:
						* (+) Action [Diabetes: Bad -> Good] (2/16 = 12.5% / MeanCost = 0.06

### Iteration: 70
#### Before:
- If SkinThickness<27:
	- If SkinThickness<18:
		- If Pregnancies<2:
			* Action [Diabetes: Bad -> Good] (0/94 = 0.0% / MeanCost = 0.0):
		- Else:
			* Action [Diabetes: Bad -> Good] (174/221 = 78.7% / MeanCost = 0.67):
				* Pregnancies: +2
				* Glucose: +5
				* SkinThickness: +10
				* BMI: +38.6880
				* DiabetesPedigreeFunction: +0.2282
				* Age: +8
	- Else:
		* Action [Diabetes: Bad -> Good] (170/172 = 98.8% / MeanCost = 0.682):
			* Pregnancies: +5
			* Glucose: +36
			* SkinThickness: +73
			* BMI: +38.6880
			* DiabetesPedigreeFunction: +0.3647
			* Age: +11
- Else:
	- If Pregnancies<1:
		* Action [Diabetes: Bad -> Good] (49/57 = 86.0% / MeanCost = 0.318):
			* Pregnancies: +1
			* Glucose: +15
			* BloodPressure: -6
			* SkinThickness: +39
			* BMI: +34.7080
			* DiabetesPedigreeFunction: +0.0827
			* Age: +3
	- Else:
		- If BMI<37.9:
			- If Age<23:
				- If BMI<30.1:
					* Action [Diabetes: Bad -> Good] (13/13 = 100.0% / MeanCost = 0.708):


- If SkinThickness<27:
	- If SkinThickness<18:
		- If Insulin<70:
			- If Pregnancies<1:
				* Action [Diabetes: Bad -> Good] (7/32 = 21.9% / MeanCost = 0.128):
					* Glucose: +8
					* BMI: +3.2240
					* Age: +2
			- Else:
				- If Glucose<124:
					* Action [Diabetes: Bad -> Good] (168/173 = 97.1% / MeanCost = 0.728):
						* Pregnancies: +2
						* Glucose: +54
						* SkinThickness: +19
						* BMI: +38.6880
						* DiabetesPedigreeFunction: +0.3687
						* Age: +2
				- Else:
					* Action [Diabetes: Bad -> Good] (63/63 = 100.0% / MeanCost = 0.693):
						* Pregnancies: +5
						* Glucose: +5
						* SkinThickness: +11
						* BMI: +33.8520
						* DiabetesPedigreeFunction: +0.2300
						* Age: -1
		- Else:
			* Action [Diabetes: Bad -> Good] (47/47 = 100.0% / MeanCost = 0.745):
				* Pregnancies: +3
				* Glucose: +6
				* BloodPressure: -21
				* SkinThickness: +22
				* Insulin: -70
				* BMI: +45.0800
				* DiabetesPedigreeFunction: +0.5331
				* Age: +12
	- Else:
		* Action

...

- If DiabetesPedigreeFunction<0.2242:
	- If BMI<30.1:
		- If Insulin<155:
			- If SkinThickness<18:
				* Action [Diabetes: Bad -> Good] (0/61 = 0.0% / MeanCost = 0.0):
			- Else:
				* Action [Diabetes: Bad -> Good] (29/29 = 100.0% / MeanCost = 0.722):
					* Pregnancies: +5
					* Glucose: +53
					* SkinThickness: +31
					* BMI: +47.5160
					* DiabetesPedigreeFunction: +0.4076
					* Age: +15
		- Else:
			* Action [Diabetes: Bad -> Good] (8/8 = 100.0% / MeanCost = 0.803):
				* Pregnancies: +8
				* Glucose: +60
				* BloodPressure: -60
				* SkinThickness: +66
				* Insulin: -168
				* BMI: +19.8120
				* DiabetesPedigreeFunction: +0.5719
				* Age: +26
	- Else:
		* Action [Diabetes: Bad -> Good] (73/78 = 93.6% / MeanCost = 0.375):
			* Pregnancies: +1
			* Glucose: +26
			* BloodPressure: -7
			* SkinThickness: +13
			* BMI: +35.9920
			* DiabetesPedigreeFunction: +0.1305
			* Age: +6
- Else:
	- If Insulin<155:
		- If Pregnancies<6:
			- If Glucose<94:
				- If BMI<30.1:
					- 

- If DiabetesPedigreeFunction<0.2242:
	- If BMI<30.1:
		- If Insulin<155:
			- If SkinThickness<18:
				- If BloodPressure<82:
					* Action [Diabetes: Bad -> Good] (0/54 = 0.0% / MeanCost = 0.0):
				- Else:
					* Action [Diabetes: Bad -> Good] (5/7 = 71.4% / MeanCost = 0.276):
						* Pregnancies: +2
						* Glucose: +27
						* BloodPressure: -11
						* BMI: +3.3920
						* DiabetesPedigreeFunction: +0.1084
						* Age: +10
			- Else:
				* Action [Diabetes: Bad -> Good] (29/29 = 100.0% / MeanCost = 0.722):
					* Pregnancies: +5
					* Glucose: +53
					* SkinThickness: +31
					* BMI: +47.5160
					* DiabetesPedigreeFunction: +0.4076
					* Age: +15
		- Else:
			* Action [Diabetes: Bad -> Good] (8/8 = 100.0% / MeanCost = 0.803):
				* Pregnancies: +8
				* Glucose: +60
				* BloodPressure: -60
				* SkinThickness: +66
				* Insulin: -168
				* BMI: +19.8120
				* DiabetesPedigreeFunction: +0.5719
				* Age: +26
	- Else:
		* Action [Diabetes: Bad -> Good] (73/78 = 93.6% / MeanCos

### Iteration: 450
#### Before:
- If DiabetesPedigreeFunction<0.2242:
	- If BMI<30.1:
		- If Insulin<155:
			- If Glucose<146:
				- If BloodPressure<82:
					* Action [Diabetes: Bad -> Good] (74/74 = 100.0% / MeanCost = 0.796):
						* Pregnancies: +3
						* Glucose: +58
						* SkinThickness: +23
						* BMI: +49.9720
						* DiabetesPedigreeFunction: +0.3747
						* Age: +12
				- Else:
					* Action [Diabetes: Bad -> Good] (5/7 = 71.4% / MeanCost = 0.276):
						* Pregnancies: +2
						* Glucose: +27
						* BloodPressure: -11
						* BMI: +3.3920
						* DiabetesPedigreeFunction: +0.1084
						* Age: +10
			- Else:
				* Action [Diabetes: Bad -> Good] (9/9 = 100.0% / MeanCost = 0.68):
					* Pregnancies: +9
					* Glucose: +5
					* BloodPressure: -19
					* SkinThickness: +19
					* BMI: +17.6880
					* DiabetesPedigreeFunction: +0.3706
					* Age: +18
		- Else:
			* Action [Diabetes: Bad -> Good] (8/8 = 100.0% / MeanCost = 0.803):
				* Pregnancies: +8
				* Glucose: +60
				* 

- If SkinThickness<27:
	- If Insulin<155:
		- If Glucose<146:
			- If BloodPressure<82:
				* Action [Diabetes: Bad -> Good] (352/352 = 100.0% / MeanCost = 0.735):
					* Pregnancies: +2
					* Glucose: +56
					* SkinThickness: +23
					* BMI: +35.4640
					* DiabetesPedigreeFunction: +0.5152
					* Age: +9
			- Else:
				- (+) If SkinThickness<18:
					* (+) Action [Diabetes: Bad -> Good] (32/39 = 82.1% / MeanCost = 0.576):
						* Pregnancies: +2
						* Glucose: +17
						* BloodPressure: -8
						* BMI: +38.6880
						* DiabetesPedigreeFunction: +0.2489
						* Age: +12
				- Else:
					* (+) Action [Diabetes: Bad -> Good] (16/18 = 88.9% / MeanCost = 0.503):
						* Pregnancies: +2
						* Glucose: +24
						* BloodPressure: -11
						* SkinThickness: +6
						* BMI: +41.3920
						* DiabetesPedigreeFunction: +0.1692
						* Age: +7
		- Else:
			* Action [Diabetes: Bad -> Good] (36/36 = 100.0% / MeanCost = 0.644):
				* Pregnancies: +7
				* Glucose: +5
				* SkinThickness: +14
		

### Iteration: 490
#### Before:
- If BMI<26.2:
	- If Insulin<155:
		- If SkinThickness<18:
			- If Glucose<94:
				- If Age<23:
					* Action [Diabetes: Bad -> Good] (0/8 = 0.0% / MeanCost = 0.0):
				- Else:
					* Action [Diabetes: Bad -> Good] (0/24 = 0.0% / MeanCost = 0.0):
			- Else:
				- If Pregnancies<2:
					- If DiabetesPedigreeFunction<0.455:
						- If Pregnancies<1:
							* Action [Diabetes: Bad -> Good] (17/17 = 100.0% / MeanCost = 0.903):
								* Pregnancies: +9
								* Glucose: +62
								* SkinThickness: +37
								* BMI: +56.4200
								* DiabetesPedigreeFunction: +2.0141
								* Age: +30
						- Else:
							* Action [Diabetes: Bad -> Good] (0/11 = 0.0% / MeanCost = 0.0):
					- Else:
						* Action [Diabetes: Bad -> Good] (7/7 = 100.0% / MeanCost = 0.597):
							* Pregnancies: +5
							* Glucose: +69
							* BloodPressure: -17
							* SkinThickness: +24
							* BMI: +11.2280
							* DiabetesPedigreeFunction: +1.7606
							* Age: +16
				- Else:
				

In [31]:
x_cet = cet.predict(X_vl) + X_vl
y_cet = mdl.predict(x_cet)

print('Negative accuracy of CET = {}'.format(np.mean(y_cet != mdl.predict(X_vl))))
print('Negative Oulier CET = {}'.format(np.mean(isolation.predict(x_cet) == 1)))

Negative accuracy of CET = 1.0
Negative Oulier CET = 0.17142857142857143


In [33]:
print('Sparsity', np.mean(np.sum(x_cet-X_vl!=0, axis=1)))

Sparsity 5.662857142857143


In [34]:
cet.print_tree()

- If Glucose<108:
	* Action [Diabetes: Bad -> Good] (409/409 = 100.0% / MeanCost = 0.682):
		* Pregnancies: +2
		* Glucose: +54
		* SkinThickness: +28
		* BMI: +33.8520
		* DiabetesPedigreeFunction: +0.4684
		* Age: +13
- Else:
	- If SkinThickness<35:
		- If BMI<33.7:
			* Action [Diabetes: Bad -> Good] (241/243 = 99.2% / MeanCost = 0.703):
				* Pregnancies: +3
				* Glucose: +5
				* SkinThickness: +19
				* BMI: +48.3600
				* DiabetesPedigreeFunction: +0.4076
		- Else:
			* Action [Diabetes: Bad -> Good] (56/56 = 100.0% / MeanCost = 0.23):
				* Glucose: +16
				* BloodPressure: -2
				* SkinThickness: +3
				* BMI: +35.6160
				* DiabetesPedigreeFunction: +0.0368
				* Age: +2
	- Else:
		* Action [Diabetes: Bad -> Good] (70/70 = 100.0% / MeanCost = 0.251):
			* Glucose: +11
			* BloodPressure: -3
			* SkinThickness: +42
			* BMI: +23.4240
			* DiabetesPedigreeFunction: +0.0248
			* Age: +1

